In [34]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge

from sklearn import metrics



In [35]:
##### Loading ######
dfTrain = pd.read_csv('datasets/train.csv', delimiter=',')
X = dfTrain.iloc[:,1:]
Y = dfTrain.iloc[:,1]
#print(X,Y)

In [42]:
# I need to do also the RMSE computation, I don't know how with this
#clf = RidgeCV(alphas=[0.1,1,10,100,200],cv=10, scoring = 'neg_root_mean_squared_error').fit(X, Y)

In [37]:
## Split in K-folds
kf = KFold(n_splits=10, shuffle = True)

In [38]:
# Convert to numpy arrays to use indices from KFold later on
X = X.to_numpy()
Y = Y.to_numpy()

In [40]:
# Regularization parameters
reg_params = [0.1, 1, 10, 100, 200]

# Dictionary to store RMSE for each regularization parameter
RMSE = {}

# Loop over all regularisation parameters to test
for reg_param in reg_params:
    
    # Initialize dictionary to store RMSE for each fold as well as avg and std
    RMSE[reg_param] = {"list": [], "avg": 0., "std": 0.}

    # We have to split each time! Cannot loop more than once over kf.split...
    kf_indices = kf.split(X)
    for train_indices, validation_indices in kf_indices:
        #print(validation_indices)
        # Initialize a Ridge regression model
        ridge_regressor = Ridge(alpha=reg_param)
        # Fit it on the training part of the fold
        ridge_regressor.fit(X[train_indices], Y[train_indices])
        # Evaluate on the rest of the dataset
        y_validation_predictions = ridge_regressor.predict(X[validation_indices])
        # Calculate and store RMSE
        RMSE[reg_param]["list"].append(np.sqrt(metrics.mean_squared_error(Y[validation_indices], y_validation_predictions)))

    # Compute average RMSE and std
    RMSE[reg_param]["avg"] = np.mean(RMSE[reg_param]["list"])
    RMSE[reg_param]["std"] = np.std(RMSE[reg_param]["list"], ddof=1)
    #print(np.mean(RMSE[reg_param]["list"]),RMSE[reg_param]["avg"])

In [41]:
file = open("to_submit.txt", "w") 

for param in RMSE.keys():
    print("lambda = %.1e" %param)
    print("\tavg = %.3f   std = %.3f" %(RMSE[param]["avg"], RMSE[param]["std"]))
    file.write(str(RMSE[param]["avg"])+"\n")
    
file.close()

lambda = 1.0e-01
	avg = 0.000   std = 0.000
lambda = 1.0e+00
	avg = 0.001   std = 0.002
lambda = 1.0e+01
	avg = 0.021   std = 0.042
lambda = 1.0e+02
	avg = 0.128   std = 0.153
lambda = 2.0e+02
	avg = 0.247   std = 0.294
